In [62]:
import glob
import pandas as pd

In [63]:
raw_path = '../data/raw/'

# 1. Update standalone files with the new encoding
calendar = pd.read_csv(f'{raw_path}AdventureWorks_Calendar.csv', encoding='latin-1')
customers = pd.read_csv(f'{raw_path}AdventureWorks_Customers.csv', encoding='latin-1')
products = pd.read_csv(f'{raw_path}AdventureWorks_Products.csv', encoding='latin-1')
returns = pd.read_csv(f'{raw_path}AdventureWorks_Returns.csv', encoding='latin-1')
territories = pd.read_csv(f'{raw_path}AdventureWorks_Territories.csv', encoding='latin-1')
product_categories = pd.read_csv(f'{raw_path}AdventureWorks_Product_Categories.csv', encoding='latin-1')
product_subcategories = pd.read_csv(f'{raw_path}AdventureWorks_Product_Subcategories.csv', encoding='latin-1')

# 2. Update the Sales merge
sales_paths = glob.glob(f'{raw_path}AdventureWorks_Sales_*.csv')
sales = pd.concat((pd.read_csv(f, encoding='latin-1') for f in sales_paths), ignore_index=True)

In [64]:
# Create a dictionary of all your dataframes
dataframes = {
    "Calendar": calendar,
    "Customers": customers,
    "Products": products,
    "Returns": returns,
    "Territories": territories,
    "Product_Categories": product_categories,
    "Product_Subcategories": product_subcategories,
    "Sales": sales
}

# Print the columns for each
for name, df in dataframes.items():
    print(f"{name}: {list(df.columns)}")

Calendar: ['Date']
Customers: ['CustomerKey', 'Prefix', 'FirstName', 'LastName', 'BirthDate', 'MaritalStatus', 'Gender', 'EmailAddress', 'AnnualIncome', 'TotalChildren', 'EducationLevel', 'Occupation', 'HomeOwner']
Products: ['ProductKey', 'ProductSubcategoryKey', 'ProductSKU', 'ProductName', 'ModelName', 'ProductDescription', 'ProductColor', 'ProductSize', 'ProductStyle', 'ProductCost', 'ProductPrice']
Returns: ['ReturnDate', 'TerritoryKey', 'ProductKey', 'ReturnQuantity']
Territories: ['SalesTerritoryKey', 'Region', 'Country', 'Continent']
Product_Categories: ['ProductCategoryKey', 'CategoryName']
Product_Subcategories: ['ProductSubcategoryKey', 'SubcategoryName', 'ProductCategoryKey']
Sales: ['OrderDate', 'StockDate', 'OrderNumber', 'ProductKey', 'CustomerKey', 'TerritoryKey', 'OrderLineItem', 'OrderQuantity']


# EDA

**clean calendar**

In [65]:
print('duplicates:',calendar.duplicated().sum())
print('shape:',calendar.shape)
print('na:',calendar.isna().sum())

duplicates: 0
shape: (912, 1)
na: Date    0
dtype: int64


In [66]:
# 1. Convert to datetime objects to ensure proper math
calendar['Date'] = pd.to_datetime(calendar['Date'])

# 2. Define the expected range
start_date = '2015-01-01'
end_date = '2017-06-30'
expected_days = pd.date_range(start=start_date, end=end_date)

# 3. Validation Logic
actual_unique_days = calendar['Date'].nunique()
missing_days = len(expected_days) - actual_unique_days

print(f"Expected days: {len(expected_days)}")
print(f"Actual unique days: {actual_unique_days}")

if missing_days == 0:
    print("✅ Success: The calendar is complete with no gaps.")
else:
    print(f"⚠️ Warning: There are {missing_days} days missing.")

Expected days: 912
Actual unique days: 912
✅ Success: The calendar is complete with no gaps.


In [67]:
#pyscript: ensure the dates are in the format (d/m/yr) and of type datetime
# ensure any nan or weird values are handled in try catch block.
# ensure no value in the past are inserted (val<min_date)= ignore

**cleaning customers**

In [68]:
customers.head()

,CustomerKey,Prefix,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner
0,11000,MR.,JON,YANG,4/8/1966,M,M,jon24@adventure-works.com,"$90,000",2,Bachelors,Professional,Y
1,11001,MR.,EUGENE,HUANG,5/14/1965,S,M,eugene10@adventure-works.com,"$60,000",3,Bachelors,Professional,N
2,11002,MR.,RUBEN,TORRES,8/12/1965,M,M,ruben35@adventure-works.com,"$60,000",3,Bachelors,Professional,Y
3,11003,MS.,CHRISTY,ZHU,2/15/1968,S,F,christy12@adventure-works.com,"$70,000",0,Bachelors,Professional,N
4,11004,MRS.,ELIZABETH,JOHNSON,8/8/1968,S,F,elizabeth5@adventure-works.com,"$80,000",5,Bachelors,Professional,Y


In [69]:
print('duplicates:',customers.duplicated().sum())
print('shape:',customers.shape)
print('na:\n',customers.isna().sum())

duplicates: 0
shape: (18148, 13)
na:
 CustomerKey         0
Prefix            130
FirstName           0
LastName            0
BirthDate           0
MaritalStatus       0
Gender            130
EmailAddress        0
AnnualIncome        0
TotalChildren       0
EducationLevel      0
Occupation          0
HomeOwner           0
dtype: int64


In [70]:
# Create a mask for rows where BOTH Prefix and Gender are missing
both_missing = customers[customers['Prefix'].isna() & customers['Gender'].isna()]

print(f"Rows where both are missing: {len(both_missing)}")
print(f"Rows where ONLY Prefix is missing: {customers['Prefix'].isna().sum() - len(both_missing)}")

Rows where both are missing: 130
Rows where ONLY Prefix is missing: 0


In [71]:
import gender_guesser.detector as gender
# 1. Initialize detector
detector = gender.Detector()

def clean_customers_full(df):
    customers_clean = df.copy()
    
    # 2. Define the logic for guessing and prefixing
    def impute_logic(row):
        g = row['Gender']
        p = row['Prefix']
        
        # Step A: Guess Gender if missing
        if pd.isna(g):
            guess = detector.get_gender(row['FirstName'])
            if 'female' in guess:
                g = 'F'
            elif 'male' in guess:
                g = 'M'
        
        # Step B: Determine Prefix based on Gender + Marital Status
        if pd.isna(p):
            if g == 'M':
                p = 'MR.'
            elif g == 'F':
                # M = Married (Mrs), S = Single (Ms)
                p = 'MRS.' if row['MaritalStatus'] == 'M' else 'MS.'
        
        return pd.Series([g, p])

    # 3. Apply logic to the DataFrame
    customers_clean[['Gender', 'Prefix']] = customers_clean.apply(impute_logic, axis=1)
    
    # 4. Final Fallback: Fill remaining 130 if still NaN (unknown names)
    # Filling with 'U' for Gender and 'MR.' (most common) for Prefix
    customers_clean['Gender'] = customers_clean['Gender'].fillna('U')
    customers_clean['Prefix'] = customers_clean['Prefix'].fillna(customers_clean['Prefix'].mode()[0])
    
    return customers_clean

# Execute
customers_clean = clean_customers_full(customers)

# Validation check
print(customers_clean[['Prefix', 'Gender']].isna().sum())

Prefix    0
Gender    0
dtype: int64


In [72]:
customers_clean.dtypes

CustomerKey        int64
Prefix            object
FirstName            str
LastName             str
BirthDate            str
MaritalStatus        str
Gender            object
EmailAddress         str
AnnualIncome         str
TotalChildren      int64
EducationLevel       str
Occupation           str
HomeOwner            str
dtype: object

In [73]:
customers_clean.head(3)

,CustomerKey,Prefix,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner
0,11000,MR.,JON,YANG,4/8/1966,M,M,jon24@adventure-works.com,"$90,000",2,Bachelors,Professional,Y
1,11001,MR.,EUGENE,HUANG,5/14/1965,S,M,eugene10@adventure-works.com,"$60,000",3,Bachelors,Professional,N
2,11002,MR.,RUBEN,TORRES,8/12/1965,M,M,ruben35@adventure-works.com,"$60,000",3,Bachelors,Professional,Y


In [74]:
def transform_customer_data(df):
    # Create a local copy to avoid modifying the original dataframe accidentally
    df_clean = df.copy()

    # 1. Strip spaces and convert to Category
    cat_cols = ['Prefix', 'MaritalStatus', 'Gender', 'EducationLevel', 'HomeOwner', 'Occupation']
    for col in cat_cols:
        df_clean[col] = df_clean[col].str.strip().astype('category')

    # 2. Clean and convert AnnualIncome
    df_clean['AnnualIncome'] = (
        df_clean['AnnualIncome']
        .replace(r'[\$,]', '', regex=True)
        .astype(float)
    )

    # 3. Convert BirthDate (Fixed the variable name to df_clean)
    df_clean['BirthDate'] = pd.to_datetime(
        df_clean['BirthDate'], 
        format='mixed'
    )
    
    return df_clean

# Execute
customers_clean = transform_customer_data(customers_clean)

In [75]:
customers_clean.describe().round(2)

,CustomerKey,BirthDate,AnnualIncome,TotalChildren
count,18148.00,18148,18148.00,18148.00
mean,20247.47,1962-01-29 07:41:05.329513,57269.12,1.84
min,11000.00,1910-08-13 00:00:00,10000.00,0.00
25%,15626.75,1954-09-11 18:00:00,30000.00,0.00
50%,20252.50,1963-08-20 00:00:00,60000.00,2.00
75%,24868.25,1970-09-26 00:00:00,70000.00,3.00
max,29483.00,1980-12-26 00:00:00,170000.00,5.00
std,5336.25,NaN,32236.54,1.61


In [76]:
customers_clean.head(3)

,CustomerKey,Prefix,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner
0,11000,MR.,JON,YANG,1966-04-08,M,M,jon24@adventure-works.com,90000.0,2,Bachelors,Professional,Y
1,11001,MR.,EUGENE,HUANG,1965-05-14,S,M,eugene10@adventure-works.com,60000.0,3,Bachelors,Professional,N
2,11002,MR.,RUBEN,TORRES,1965-08-12,M,M,ruben35@adventure-works.com,60000.0,3,Bachelors,Professional,Y


In [77]:
#ensure name/last name are actual string letters don't involve numerical
# ensure prefix within the 3 unique vals
# ensure birthdate is valid and within the 100 yr old range
# ensure maritalstatus within the two vals range
# gender too
# ensure valid email format
# ensure total child don't exceed 15
# ensure education/occupation/homeower are within their values
# ensure annual income is in a valid range of 99% of dist

**products clean**

In [81]:
print('duplicates:',products.duplicated().sum())
print('shape:',products.shape)
print('na:\n',products.isna().sum())

duplicates: 0
shape: (293, 11)
na:
 ProductKey                0
ProductSubcategoryKey     0
ProductSKU                0
ProductName               0
ModelName                 0
ProductDescription        0
ProductColor             50
ProductSize               0
ProductStyle              0
ProductCost               0
ProductPrice              0
dtype: int64


In [82]:
products.head(3)

,ProductKey,ProductSubcategoryKey,ProductSKU,ProductName,ModelName,ProductDescription,ProductColor,ProductSize,ProductStyle,ProductCost,ProductPrice
0,214,31,HL-U509-R,"Sport-100 Helmet, Red",Sport-100,"Universal fit, well-vented, lightweight , snap...",Red,0,0,13.0863,34.9900
1,215,31,HL-U509,"Sport-100 Helmet, Black",Sport-100,"Universal fit, well-vented, lightweight , snap...",Black,0,0,12.0278,33.6442
2,218,23,SO-B909-M,"Mountain Bike Socks, M",Mountain Bike Socks,Combination of natural and synthetic fibers st...,White,M,U,3.3963,9.5000


In [87]:
products.dtypes

ProductKey                 int64
ProductSubcategoryKey      int64
ProductSKU                   str
ProductName                  str
ModelName                    str
ProductDescription           str
ProductColor                 str
ProductSize                  str
ProductStyle                 str
ProductCost              float64
ProductPrice             float64
dtype: object

In [94]:
#imputing colors:
def impute_colors(df):
    df_clean = df.copy()
    # List sorted by length descending to catch 'Silver/Black' before 'Silver'
    colors = ['Silver/Black', 'Red', 'Black', 'White', 'Blue', 'Multi', 'Silver', 'Yellow', 'Grey']
    
    # 1. Calculate Mode for fallback
    mode_color = df_clean['ProductColor'].mode()[0]
    
    # 2. Imputation Logic
    def find_color(row):
        if pd.isna(row['ProductColor']):
            for color in colors:
                if color.lower() in row['ProductName'].lower():
                    return color
            return mode_color
        return row['ProductColor']
    
    df_clean['ProductColor'] = df_clean.apply(find_color, axis=1)
    return df_clean

# products_clean = impute_colors(products)

In [95]:
products_clean= impute_colors(products)

In [102]:
def finalize_products(df):
    df_clean = df.copy()
    
    # Columns to transform
    cols_to_fix = ['ProductColor', 'ProductSize', 'ProductStyle']
    
    for col in cols_to_fix:
        # Strip whitespace and convert to category
        df_clean[col] = df_clean[col].str.strip().astype('category')
        
    return df_clean

# Execute
products_clean = finalize_products(products_clean)

In [108]:
products_clean.describe().round(2)

,ProductKey,ProductSubcategoryKey,ProductCost,ProductPrice
count,293.00,293.00,293.00,293.00
mean,441.55,12.28,413.66,714.44
std,108.06,9.89,517.61,872.16
min,214.00,1.00,0.86,2.29
25%,354.00,2.00,31.72,63.50
50%,457.00,12.00,199.38,333.42
75%,533.00,17.00,601.74,1003.91
max,606.00,37.00,2171.29,3578.27


**product subcategories cleaning**

In [105]:
product_subcategories.head(2)

,ProductSubcategoryKey,SubcategoryName,ProductCategoryKey
0,1,Mountain Bikes,1
1,2,Road Bikes,1


In [106]:
print('duplicates:',product_subcategories.duplicated().sum())
print('shape:',product_subcategories.shape)
print('na:\n',product_subcategories.isna().sum())

duplicates: 0
shape: (37, 3)
na:
 ProductSubcategoryKey    0
SubcategoryName          0
ProductCategoryKey       0
dtype: int64


In [110]:
product_subcategories['SubcategoryName'].unique()

<StringArray>
[   'Mountain Bikes',        'Road Bikes',     'Touring Bikes',
        'Handlebars',   'Bottom Brackets',            'Brakes',
            'Chains',         'Cranksets',       'Derailleurs',
             'Forks',          'Headsets',   'Mountain Frames',
            'Pedals',       'Road Frames',           'Saddles',
    'Touring Frames',            'Wheels',        'Bib-Shorts',
              'Caps',            'Gloves',           'Jerseys',
            'Shorts',             'Socks',            'Tights',
             'Vests',        'Bike Racks',       'Bike Stands',
 'Bottles and Cages',          'Cleaners',           'Fenders',
           'Helmets',   'Hydration Packs',            'Lights',
             'Locks',          'Panniers',             'Pumps',
   'Tires and Tubes']
Length: 37, dtype: str

**Product category cleaning**

In [114]:
product_categories

,ProductCategoryKey,CategoryName
0,1,Bikes
1,2,Components
2,3,Clothing
3,4,Accessories


**Territories clean**

In [118]:
territories

,SalesTerritoryKey,Region,Country,Continent
0,1,Northwest,United States,North America
1,2,Northeast,United States,North America
2,3,Central,United States,North America
3,4,Southwest,United States,North America
4,5,Southeast,United States,North America
5,6,Canada,Canada,North America
6,7,France,France,Europe
7,8,Germany,Germany,Europe
8,9,Australia,Australia,Pacific
9,10,United Kingdom,United Kingdom,Europe


**sales cleaning**

In [127]:
print('duplicates:',sales.duplicated().sum())
print('shape:',sales.shape)
print('na:\n',sales.isna().sum())

duplicates: 0
shape: (56046, 8)
na:
 OrderDate        0
StockDate        0
OrderNumber      0
ProductKey       0
CustomerKey      0
TerritoryKey     0
OrderLineItem    0
OrderQuantity    0
dtype: int64


In [128]:
sales.dtypes

OrderDate          str
StockDate          str
OrderNumber        str
ProductKey       int64
CustomerKey      int64
TerritoryKey     int64
OrderLineItem    int64
OrderQuantity    int64
dtype: object

In [129]:
sales.head(3)

,OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
0,1/1/2015,9/21/2001,SO45080,332,14657,1,1,1
1,1/1/2015,12/5/2001,SO45079,312,29255,4,1,1
2,1/1/2015,10/29/2001,SO45082,350,11455,9,1,1


In [134]:
def transform_sales_columns(df):
    df_clean = df.copy()

    # 1. Convert to datetime objects
    # We keep them as columns, not index
    date_cols = ['OrderDate', 'StockDate']
    for col in date_cols:
        df_clean[col] = pd.to_datetime(df_clean[col], format='mixed')

    # 2. Convert to Day-Month-Year String (Optional/Display)
    # Note: This makes the column a string. 
    # Usually better to keep as datetime for the 'Join' step.
    return df_clean

# Execute
sales_clean = transform_sales_columns(sales)

In [135]:
sales_clean.head()

,OrderDate,StockDate,OrderNumber,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
0,2015-01-01,2001-09-21,SO45080,332,14657,1,1,1
1,2015-01-01,2001-12-05,SO45079,312,29255,4,1,1
2,2015-01-01,2001-10-29,SO45082,350,11455,9,1,1
3,2015-01-01,2001-11-16,SO45081,338,26782,6,1,1
4,2015-01-02,2001-12-15,SO45083,312,14947,10,1,1


In [139]:
# 1. Check for Inclusion (Do all sales have a calendar entry?)
missing_dates = sales_clean[~sales_clean['OrderDate'].isin(calendar['Date'])]

# 2. Check for Equivalence (Do the start/end ranges match exactly?)
sales_range = (sales_clean['OrderDate'].min(), sales_clean['OrderDate'].max())
calendar_range = (calendar['Date'].min(), calendar['Date'].max())

print(f"Sales Range: {sales_range}")
print(f"Calendar Range: {calendar_range}")
print(f"Dates in Sales not in Calendar: {len(missing_dates)}")

Sales Range: (Timestamp('2015-01-01 00:00:00'), Timestamp('2017-06-30 00:00:00'))
Calendar Range: (Timestamp('2015-01-01 00:00:00'), Timestamp('2017-06-30 00:00:00'))
Dates in Sales not in Calendar: 0


In [140]:
sales.describe().round(2)

,ProductKey,CustomerKey,TerritoryKey,OrderLineItem,OrderQuantity
count,56046.00,56046.00,56046.00,56046.00,56046.00
mean,438.96,18843.65,6.25,1.90,1.50
std,118.61,5412.45,2.96,1.02,0.61
min,214.00,11000.00,1.00,1.00,1.00
25%,360.00,14016.00,4.00,1.00,1.00
50%,479.00,18157.00,7.00,2.00,1.00
75%,529.00,23425.75,9.00,2.00,2.00
max,606.00,29483.00,10.00,8.00,3.00


**returns clean**

In [142]:
print('duplicates:',returns.duplicated().sum())
print('shape:',returns.shape)
print('na:\n',returns.isna().sum())

duplicates: 0
shape: (1809, 4)
na:
 ReturnDate        0
TerritoryKey      0
ProductKey        0
ReturnQuantity    0
dtype: int64


In [143]:
returns.dtypes

ReturnDate          str
TerritoryKey      int64
ProductKey        int64
ReturnQuantity    int64
dtype: object

In [144]:
returns.head(3)

,ReturnDate,TerritoryKey,ProductKey,ReturnQuantity
0,1/18/2015,9,312,1
1,1/18/2015,10,310,1
2,1/21/2015,8,346,1


In [146]:
returns.describe().round(2)

,TerritoryKey,ProductKey,ReturnQuantity
count,1809.00,1809.00,1809.00
mean,6.26,443.46,1.01
std,2.95,115.33,0.10
min,1.00,214.00,1.00
25%,4.00,369.00,1.00
50%,7.00,479.00,1.00
75%,9.00,529.00,1.00
max,10.00,606.00,2.00


In [147]:
# calendar / customers_clean / products_clean / product_categories
# product_subcategory / territory
# sales_clean / returns